#### 1. Installing the required python packages

In [0]:
# !python -m pip install --upgrade pip

  Using cached pip-22.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


In [0]:
# !pip install -r requirements38.txt

  Using cached absl_py-0.7.1-py3-none-any.whl
  Using cached astor-0.8.0-py2.py3-none-any.whl (27 kB)
  Using cached boto-2.49.0-py2.py3-none-any.whl (1.4 MB)
  Using cached boto3-1.9.188-py2.py3-none-any.whl (128 kB)
  Using cached botocore-1.12.188-py2.py3-none-any.whl (5.6 MB)
  Using cached certifi-2019.6.16-py2.py3-none-any.whl (157 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached cloudpickle-1.6.0-py3-none-any.whl (23 kB)
  Using cached docutils-0.14-py3-none-any.whl (543 kB)
  Using cached gast-0.2.2-py3-none-any.whl
  Using cached gdown-4.3.0-py3-none-any.whl
  Using cached gensim-3.8.0-cp39-cp39-linux_x86_64.whl
  Using cached google_pasta-0.1.7-py3-none-any.whl (52 kB)
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
  Using cached jmespath-0.9.4-py2.py3-none-any.whl (24 kB)
  Using cached joblib-1.0.0-py3-none-any.whl (302 kB)
  Using cached jupyter-1.0.0-py2.py3-none-any.whl (2.7 kB)
  Using cached Keras_Applications-1.0.8-py3-none-any.whl (

In [0]:
# !pip install azure-storage-file-datalake
# !pip install adlfs
# !pip install fsspec

  Using cached azure_storage_file_datalake-12.9.1-py3-none-any.whl (238 kB)
  Using cached azure_storage_blob-12.14.1-py3-none-any.whl (383 kB)
  Using cached msrest-0.7.1-py3-none-any.whl (85 kB)
  Using cached azure_core-1.26.2-py3-none-any.whl (173 kB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-fe66296a-825e-4ba9-b021-2c994e9e5621
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
  Using cached adlfs-2022.11.2-py3-none-any.whl (25 kB)
  Using cached fsspec-2022.11.0-py3-none-any.whl (139 kB)
  Using cached a

#### 2. Importing the required python packages

In [0]:
import multiprocessing as mp
import sys
from datetime import datetime
import ast

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ast import literal_eval
from pyarrow.parquet import ParquetFile

from sherlock import helpers
from sherlock.features.paragraph_vectors import (
    initialise_nltk,
    tagcol_paragraph_embeddings_features,
    train_paragraph_embeddings_features
)
from sherlock.features.preprocessing import convert_string_lists_to_lists
from sherlock.functional import extract_features_to_csv

In [0]:
print(f'Started at {datetime.now()}')

Started at 2023-01-07 05:43:50.120049


#### 3. Connect to Azure Data Storage

In [0]:
import os, uuid, sys
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import ContentSettings
from azure.storage.blob import BlobServiceClient

In [0]:
def initialize_storage_account(storage_account_name, storage_account_key):
    
    try:  
        global service_client

        service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
            "https", storage_account_name), credential=storage_account_key)
    
    except Exception as e:
        print(e)

In [0]:
storage_account = "legoaistorage"
storage_account_key = "vOHAjE9vOHaxqmTRxIYETQbYlPvvFpJQ7xfky8tuWBRE9E6IbfM87ERkGcqqiHfMHs+WnEt907r6+AStjIYXlA=="
initialize_storage_account(storage_account,storage_account_key)

In [0]:
### Spark Configuration
spark.conf.set("fs.azure.account.key."+ storage_account +".dfs.core.windows.net", storage_account_key)
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [0]:
prefix_path = "abfss://datascience-dataset@legoaistorage.dfs.core.windows.net/"

#### 4. Getting the required dataset from the Azure data storage

In [0]:
filepath = 'Source_Data/sherlock/raw/train_values.parquet'
filepath_upd = prefix_path + filepath
train_samples = spark.read.format("parquet").load(filepath_upd)
train_samples = train_samples.toPandas()

In [0]:
filepath = 'Source_Data/sherlock/raw/train_labels.parquet'
filepath_upd = prefix_path + filepath
train_labels = spark.read.format("parquet").load(filepath_upd)
train_labels = train_labels.toPandas()

In [0]:
train_samples = train_samples[['values']]
train_labels = train_labels[['type']]

In [0]:
data_column_name = 'values'
labels_column_name = 'type'

# train_samples = train_samples[:1000]
# train_labels = train_labels[:1000]

train_samples_converted = train_samples[data_column_name].apply(ast.literal_eval)
y_train = train_labels[labels_column_name].to_list()

---------------------------------------------------------------------------
The Python process exited with exit code 137 (SIGKILL: Killed). This may have been caused by an OOM error. Check your command's memory usage.

The last 10 KB of the process's stderr and stdout can be found below. See driver logs for full logs.
---------------------------------------------------------------------------
Last messages on stderr:
Sat Jan  7 05:41:13 2023 Connection to spark from PID  6979
Sat Jan  7 05:41:13 2023 Initialized gateway on port 40703
Sat Jan  7 05:41:13 2023 Connected to spark.
---------------------------------------------------------------------------
Last messages on stdout:
gle-pasta-0.1.7 h5py-3.7.0 identify-2.5.12 idna-2.8 jmespath-0.9.4 joblib-1.0.0 jupyter-1.0.0 jupyter-client-7.2.0 jupyter-console-6.4.4 jupyter-core-5.1.2 line_profiler-3.3.1 loky-2.9.0 matplotlib-2.2.5 nltk-3.5 nodeenv-1.7.0 numpy-1.21.5 pandas-1.2.0 pre-commit-2.8.2 protobuf-3.9.0 pyfunctional-1.4.2 python-dat

#### 5. Train Doc2Vec

In [0]:
initialise_nltk()

In [0]:
samples = train_samples_converted.dropna()
print(f'Samples: {type(samples)}, length={len(samples)}')

labels = train_labels.values.flatten()
print(f'Labels:  {type(labels)}, length={len(labels)}')

In [0]:
# ### Converting the data to string type
# samples = samples.apply(lambda x: [str(val) for val in x])

In [0]:
import random
import nltk
from nltk.corpus import stopwords
import gensim.models.doc2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing

In [0]:
def tokenise(values):
    joined = " ".join(s for s in values if len(s) >= 2)

    # stopwords need apostrophe
    filtered = "".join(
        e for e in joined if e.isalnum() or e.isspace() or e == "'"
    ).lower()

    return [
        word
        for word in nltk.word_tokenize(filtered)
        if len(word) >= 2 and word not in STOPWORDS_ENGLISH
    ]

In [0]:
def tagcol_paragraph_embeddings_features(train_data: pd.Series, train_labels: list):
    random.seed(13)

    columns = []

    for i, col in enumerate(train_data):
        label = train_labels[i]
        values = random.sample(col, min(1000, len(col)))

        if len(values) > 0:
            values = list(map(lambda s: "" if s is None else str(s), values))

        tokens = tokenise(values)

        columns.append(TaggedDocument(tokens, [label]))

    return columns

In [0]:
start = datetime.now()
STOPWORDS_ENGLISH = stopwords.words("english")

print('Tagging columns')
cols = tagcol_paragraph_embeddings_features(samples, labels)

print(f'Tagged Columns Doc2Vec Model, process took {datetime.now() - start} seconds.')

In [0]:
start = datetime.now()

vec_dim = 50
print(f'Training Doc2Vec model in {vec_dim} dimensions')

train_model = Doc2Vec(cols,dm=0,negative=3,workers=multiprocessing.cpu_count(),vector_size=vec_dim,epochs=20,min_count=2,seed=13)

print(datetime.now())

In [0]:
pwd

In [0]:
model_name = f"/sherlock/features/par_vec_trained_model_{vec_dim}.pkl"
# train_model.save(model_name)

# with open(os.path.join('sherlock','features',model_name), 'wb') as pkl:
#     pickle.dump(train_model,pkl)

# import pickle
# pickle.dump(train_model, open(os.path.join('sherlock','features',model_name), 'wb'))
# pickled_model = pickle.load(os.path.join('sherlock','features',model_name), 'rb'))

In [0]:
# sample = pd.DataFrame({'id':[1,2,3,4],'name':['s','s','d','f']})
# sample.to_csv('sherlock/features/sample.csv',index=False)

#### 6. Prediction using Doc2Vec Model

In [0]:
doc2vec_model = Doc2Vec.load(os.path.join('/',model_name))

In [0]:
doc2vec_model.random.seed(13)
doc2vec_model.infer_vector(['sample','text'])